In [ ]:
%cd ~/work/deeplearning/

In [ ]:
from theano.sandbox import cuda
cuda.use('gpu1')
%matplotlib inline
from __future__ import print_function, division
#path="../data/2cat/sample"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink
from keras.preprocessing import image, sequence
import os, sys, cv2
from shutil import copyfile, move
from random import shuffle

####
from keras.layers.convolutional import *
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam
import datetime

In [ ]:
data_dir="/home/asaeed9/work/data/2cat"
path="/home/asaeed9/work/data/2cat/sample/"
results_path = "/home/asaeed9/work/data/2cat/sample/results"
test_path = path + '/test/' #We use all the test data
global last_file_timestamp
%cd ../data/2cat/train

In [1]:
#grey images - minimal image augmentation

    
    #clean previous data
#     os.chdir(path + 'test')
    #adjust_prev_data()
    
    #build validation set
#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(validation): os.rename(shuf[i], '../valid/' + shuf[i]) 
    
#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(train): os.rename(shuf[i], '../sample/train/' + shuf[i]) 
#     %mv ../sample/train/cat*.jpg ../sample/train/cats/
#     %mv ../sample/train/dog*.jpg ../sample/train/dogs/
    
#     %cd ../valid

#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(validation): os.rename(shuf[i], '../sample/valid/' + shuf[i]) 
#     %mv ../sample/valid/cat*.jpg ../sample/valid/cats/
#     %mv ../sample/valid/dog*.jpg ../sample/valid/dogs/
#     %cd $data_dir/train  


def adjust_prev_data_sample():
    %mv $path/train/cats/* $data_dir/train/
    %mv $path/train/dogs/* $data_dir/train/
    %mv $path/valid/cats/* $data_dir/train/
    %mv $path/valid/dogs/* $data_dir/train/

#clean previous data
def adjust_prev_data():
    %mv $data_dir/valid/* $data_dir/train/    
    adjust_prev_data_sample()

#move training images
def move_from_test(new_trainset, path, train, validation):
    print(new_trainset[:10])
    
def move_to_test(images):
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(images): os.rename(shuf[i], '../sample/test/' + shuf[i]) 
    %mv ../sample/test/cat*.jpg ../sample/test/cats/
    %mv ../sample/test/dog*.jpg ../sample/test/dogs/      

    
def handle_null(train, validation):
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(train): copyfile(shuf[i], '../sample/train/' + shuf[i]) 
    %mv ../sample/train/cat*.jpg ../sample/train/cats/
    %mv ../sample/train/dog*.jpg ../sample/train/dogs/
    
    %cd ../valid

    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(validation): copyfile(shuf[i], '../sample/valid/' + shuf[i]) 
    %mv ../sample/valid/cat*.jpg ../sample/valid/cats/
    %mv ../sample/valid/dog*.jpg ../sample/valid/dogs/
    %cd $data_dir/train
    
#copy training images
def copy_samples(train, validation):
    #print("Copying new samples for training...")
    #clean previous data
    adjust_prev_data()
    
    #build validation set
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(validation): os.rename(shuf[i], '../valid/' + shuf[i]) 
    
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(train): copyfile(shuf[i], '../sample/train/' + shuf[i]) 
    %mv ../sample/train/cat*.jpg ../sample/train/cats/
    %mv ../sample/train/dog*.jpg ../sample/train/dogs/
    
    %cd ../valid

    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(validation): copyfile(shuf[i], '../sample/valid/' + shuf[i]) 
    %mv ../sample/valid/cat*.jpg ../sample/valid/cats/
    %mv ../sample/valid/dog*.jpg ../sample/valid/dogs/
    %cd $data_dir/train

def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

def pred_batch(imgs, classes):
    preds = model.predict(imgs)
    idxs = np.argmax(preds, axis=1)

    print('Shape: {}'.format(preds.shape))
    print('First 5 classes: {}'.format(classes[:5]))
    print('First 5 probabilities: {}\n'.format(preds[:5]))
    print('Predictions prob/class: ')
    
    for i in range(len(idxs)):
        idx = idxs[i]
        print ('  {:.4f}/{}'.format(preds[i, idx], classes[idx]))


def generate_size_graph(fig_no, training_size, accuracy, loss, start_size, end_size):
    plt.figure(fig_no,figsize=(7,5))
    plt.plot(training_size,accuracy)
    plt.plot(training_size,loss)
    plt.xlabel('Training Size')
    plt.ylabel('Accuracy/Loss')
    plt.title('Training Size vs Accuracy/Loss')
    plt.grid(True)
    plt.legend(['Accuracy','Loss'])
    plt.style.use(['classic'])
    plt.show()
    plt.savefig(path + '/batch_graphs/' +  str(start_size) + '_' + str(end_size) + '.jpg') 
        
def generate_graph(fig_no, epochs, train, val, label, train_title, val_title, train_size):
    plt.figure(fig_no,figsize=(7,5))
    plt.plot(epochs,train)
    plt.plot(epochs,val)
    plt.xlabel('num of Epochs')
    plt.ylabel(label)
    plt.title(train_title + ' vs ' + val_title + '( Samples:' + str(train_size) + ')')
    plt.grid(True)
    plt.legend(['train','val'])
    plt.style.use(['classic'])
    plt.show()
    plt.savefig(results_path + '/batch_graphs/' +  label + '_' + str(train_size) + '.jpg') 
    
def get_train_model(tr_batches, val_batches, epoch):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3, 256,256)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
#             Convolution2D(64,3,3, activation='relu'),
#             BatchNormalization(axis=1),
            #MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Flatten(),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(2, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=epoch - 2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model 

def train_model(model, tr_batches, val_batches, epoch):
    if not model:
        model = Sequential([
                BatchNormalization(axis=1, input_shape=(3, 256,256)),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Dropout(0.2),
    #             Convolution2D(64,3,3, activation='relu'),
    #             BatchNormalization(axis=1),
                #MaxPooling2D((3,3)),
                Convolution2D(64,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Dropout(0.2),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Dropout(0.2),
                Flatten(),
                Dense(1024, activation='relu'),
                BatchNormalization(),
                Dropout(0.2),
                Dense(2, activation='softmax')
            ])
        model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
       
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=epoch - 2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
        
    return model 
    
    
def get_test_model():
    model = Sequential([
                BatchNormalization(axis=1, input_shape=(3,256,256)),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
    #             Convolution2D(64,3,3, activation='relu'),
    #             BatchNormalization(axis=1),
                #MaxPooling2D((3,3)),
                Convolution2D(64,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Flatten(),
                Dense(1024, activation='relu'),
                BatchNormalization(),
                Dense(2, activation='softmax')
            ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])  
    
    return model

def fit(samples_copied, old_model, path, results_path, nepoch, batch_size, train_size, valid_size):
    gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

    #for train in training_range:
    model = None
    if not samples_copied:
        copy_samples(train_size, valid_size)

    tr_batches = gen_t.flow_from_directory(path + 'train', batch_size=batch_size)
    val_batches = gen_t.flow_from_directory(path + 'valid', class_mode='categorical', shuffle=True, batch_size=batch_size * 2)
    
    if old_model:
        model = train_model(old_model, tr_batches, val_batches, nepoch)
    else:
        model = train_model(None, tr_batches, val_batches, nepoch)
        
    #model.save_weights(results_path+ '/' + 'ft_' + str(train_size) + '.e' + str(nepoch))
    last_file_timestamp = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())
    #print('File Time Stamp:{}'.format(last_file_timestamp))
    model.save_weights(results_path+'/ft_{}'.format(last_file_timestamp))
    #model.load_weights(results_path+'/ft_{}'.format(last_file_timestamp))
                          
    return model, last_file_timestamp

def predict(path, model):
    gen_test = image.ImageDataGenerator()
    test_batches = gen_test.flow_from_directory(path+'test', class_mode=None, target_size=(256,256), shuffle=False, batch_size=1)
    test_data = np.concatenate([test_batches.next() for i in range(test_batches.nb_sample)])
    test_labels = onehot(test_batches.classes)
    score = model.evaluate(test_data, test_labels)
    
    probs = model.predict(test_data)
    
    #loss_score.append(score[0])
    #accuracy_score.append(score[1])
    
    print("\nLoss:{}, Accuracy:{}".format(score[0], score[1]))
    #print("\nProbabilities:{}".format(probs))
    return probs, test_batches, score[0], score[1]

def move_samples(retrain_set,dest_path, n, limit):
    cats_copied = 0
    dogs_copied = 0
    retrain_list = list(retrain_set)
    shuffle(retrain_list)
    for fil in range(n):
        fil = retrain_list.pop()
        fil_cpy = fil[fil.find('/')+1:]

        if 'cat' in fil_cpy and cats_copied <= limit:
            os.rename(os.path.join(path + "test/cats/"+ fil_cpy), os.path.join(path + dest_path + "/cats/"+ fil_cpy))
            cats_copied+=1
        elif 'dog' in fil_cpy and dogs_copied <= limit:
            os.rename(os.path.join(path + "test/dogs/"+ fil_cpy), os.path.join(path + dest_path + "/dogs/" + fil_cpy))
            dogs_copied+=1
            
    #print("Limit:", limit)        
    #print("moved cats:", cats_copied)
    #print("moved dogs:", dogs_copied)
    #print("Retrain Length: ", len(retrain_list))
    return retrain_list, cats_copied, dogs_copied


def move_to_train(retrain_set, limit):
    cats = 0
    dogs = 0
    valid_limit = int(math.floor(.2*(limit*2)))
    train_limit = int(math.floor(.8*(limit*2)))
    
    #print(retrain_set[:10])
    print("validation set: ", valid_limit)
    retrain_list, cats_copied, dogs_copied = move_samples(retrain_set, "valid", valid_limit, limit)
    valid_size = cats_copied + dogs_copied
    print("Train set: ", train_limit)
    retrain_list, cats_copied, dogs_copied = move_samples(retrain_list, "train", train_limit, limit)
    train_size = cats_copied + dogs_copied
    
    return train_size, valid_size, train_size + valid_size

def refil_test(nimages):
    os.chdir("/home/asaeed9/work/data/2cat/train")
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(nimages): os.rename(shuf[i], '../sample/test/' + shuf[i])
    #os.chdir("../sample/test/")
    #move("cat*.jpg")
    %mv ../sample/test/cat*.jpg ../sample/test/cats/
    %mv ../sample/test/dog*.jpg ../sample/test/dogs/     

In [ ]:
%cd $data_dir/train

nepoch = 5
batch_size = 64
train_size = 100
running_train_size = 100
training_set_size = []
valid_size = int(math.floor(.2 * train_size))
#print('sample size: {}'.format(train_size + valid_size))
tr_model = None
loss = 0.0
loss_array = []
accuracy = 0.0
accuracy_array = []

#copy test data
#move_to_test(3000)

i=0
for i in range(100):
    print("Train Size:{}".format(train_size))
    print("Valid Size:{}".format(valid_size))
    
    if train_size == 0 or valid_size == 0: #handle null case
        handle_null(50,10)
        train_size += 50
        valid_size += 10
        
    tr_model,file_timestamp = fit(i, tr_model, path, results_path, nepoch, batch_size, train_size, valid_size)

    model = None
    model = get_test_model() 
    #model.load_weights(results_path+'/ft_' + str(train_size) + '.e' + str(nepoch))
    #print('{0}/ft_{1}'.format(last_file_timestamp))
    #print('Last File Timestamp- before loading:{}'.format(file_timestamp))
    model.load_weights(results_path+'/ft_{}'.format(file_timestamp))
    
    probs, test_batches,loss, accuracy = predict(path, model)
    training_set_size.append(running_train_size)
    loss_array.append(loss)
    accuracy_array.append(accuracy)

    retrain_idx = np.unique(np.where(np.logical_and(probs >=0.4, probs<=0.7))[0])
    retrain_set = [test_batches.filenames[i] for i in retrain_idx]

    os.chdir(path + 'test')
    ndog = sum('dog' in name for name in retrain_set)
    ncat =  sum('cat' in name for name in retrain_set)
    limit = min(ncat, ndog)

    # print("Total Retrain Images: {}".format(limit*2))

    #print('Images to be retrained:{}'.format(limit*2))
    #print('Dogs:{0}, Cats:{1}, Total:{2}'.format(ndog, ncat, ndog+ncat))

    # # # #remove all train/validation images
    # print("Total Retrain Images: {}".format(limit*2))
    #move existing training data to the store
    adjust_prev_data_sample()
    train_size, valid_size, copied_images = move_to_train(retrain_set, limit)
    running_train_size += train_size
    refil_test(copied_images)

    print('Training Set Size:{}'.format(training_set_size))
    print('Accuracy:{}'.format(accuracy_array))
    print('Loss:{}'.format(loss_array))

print('Training Set Size:{}'.format(training_set_size))
print('Accuracy:{}'.format(accuracy_array))
print('Loss:{}'.format(loss_array))
    


In [ ]:
Training Set Size:[100, 663, 874, 1237, 1632, 1963, 2166, 2537, 2719, 2912, 3042, 3198, 3319, 3408, 3512, 3588, 3674, 3763, 3822, 3874, 3931, 3966, 4009, 4081, 4130, 4178, 4243, 4303, 4355, 4412, 4458, 4477, 4536, 4577, 4612, 4640, 4750, 4810, 4853, 4869, 4912, 4972, 5002, 5058, 5091]
Accuracy:[0.53100000000000003, 0.58133333333333337, 0.59433333329359694, 0.64633333341280619, 0.67066666674613951, 0.68900000007947282, 0.6763333331743876, 0.68499999992052718, 0.69966666666666666, 0.7143333332538605, 0.70166666674613953, 0.70233333325386049, 0.70866666662693023, 0.70599999996026352, 0.71766666670640311, 0.71899999996026354, 0.72266666658719381, 0.72233333341280614, 0.72866666666666668, 0.72999999999999998, 0.73199999999999998, 0.73733333333333329, 0.73999999999999999, 0.74133333325386053, 0.73733333333333329, 0.745, 0.7506666666666667, 0.75166666674613958, 0.75333333325386043, 0.75166666666666671, 0.75133333325386042, 0.74099999999999999, 0.7493333333333333, 0.753, 0.75166666662693027, 0.7513333333333333, 0.75299999992052713, 0.7573333333333333, 0.76200000000000001, 0.75866666674613947, 0.76066666674613947, 0.75966666674613947, 0.76533333341280618, 0.76666666674613948, 0.76666666674613948]
Loss:[0.83619017632802328, 1.1285501725673674, 1.0656420938173929, 0.96038061285018916, 1.06677472114563, 1.2086187562942505, 0.98627383597691853, 1.2054843155145645, 1.2957861052354176, 1.3309469261566798, 1.2245938794612885, 1.3769356819788614, 1.643688627059261, 1.5487953674594561, 1.8254043706506491, 1.6061326263944309, 1.8959822314778965, 2.1850071493908763, 2.2080525672808289, 2.1062208793039123, 2.1965714106808107, 2.0418047927717367, 2.2234771291514237, 2.4729285352230073, 2.2494765572920441, 2.088531137858828, 2.0007797633757192, 2.1495063023418188, 1.9922315145010749, 2.1825128635043898, 2.2795661908459541, 2.2930807283607622, 2.1990711705709498, 2.2198720518726236, 2.2256887440606952, 1.7327371734157204, 2.1334827490147825, 2.0704540131626028, 2.077258467460672, 2.0410200945374868, 1.9071191506385803, 2.2231385400940975, 2.0167198820443204, 2.0912941036230572, 2.2853920857487244]

In [ ]:
generate_size_graph(1, training_set_size, accuracy_array, loss_array, min(training_set_size), max(training_set_size))

In [ ]:








# cats_copied=5
# dogs_copied=5
# for fil in retrain_set:
#     #print("Retrain File: {}".format(fil))
#     fil_cpy = fil[fil.find('/')+1:]
#     if 'cat' in fil and cats_copied <= to_be_copied:
#         #print(os.path.join(path + "test/cats/"+ fil_cpy))
#         #print(os.path.join(path + "test_copy/"+ fil_cpy))
#         copyfile(os.path.join(path + "test/cats/"+ fil_cpy), os.path.join(path + "test_copy/"+ fil_cpy))
#         cats_copied+=1
#     elif 'dog' in fil and dogs_copied <= to_be_copied:
#         #print(os.path.join(path + "test/dogs/"+ fil_cpy))
#         #print(os.path.join(path + "test_copy/"+ fil_cpy))              
#         copyfile(os.path.join(path + "test/dogs/"+ fil_cpy), os.path.join(path + "test_copy/"+ fil_cpy))
#         dogs_copied+=1
    
#     if cats_copied >= to_be_copied and dogs_copied >= to_be_copied:
#         #print(len(os.listdir(path + "test_copy/")))
#         print(cats_copied, dogs_copied, to_be_copied)
#         break

# print("copy done.")

#move to validation directory
#replenish validation set

#move_to_test(cats_copied+dogs_copied)



        
        
        
        
        
        
        
        
    

    
#     for fil in retrain_set:
#         fil_cpy = fil[fil.find('/')+1:]
#         if 'cat' in fil and cats_copied <= limit:
#             #print(os.path.join(path + "test/cats/"+ fil_cpy))
#             #print(os.path.join(path + "test_copy/"+ fil_cpy))
#             copyfile(os.path.join(path + "test/cats/"+ fil_cpy), os.path.join(path + "test_copy/"+ fil_cpy))
#             cats_copied+=1
#         elif 'dog' in fil and dogs_copied <= limit:
#             #print(os.path.join(path + "test/dogs/"+ fil_cpy))
#             #print(os.path.join(path + "test_copy/"+ fil_cpy))              
#             copyfile(os.path.join(path + "test/dogs/"+ fil_cpy), os.path.join(path + "test_copy/"+ fil_cpy))
#             dogs_copied+=1

#         if cats_copied >= limit and dogs_copied >= limit:
#             #print(len(os.listdir(path + "test_copy/")))
#             print(cats_copied, dogs_copied, limit)
#             break

    
    
    

# def restore_backup(source, destination):
#     %mv $source/cat*.jpg $destination/cats/
#     %mv $source/dog*.jpg $destination/dogs/
    
# def remove_to_backup(source, destination):
#     %mv $source/* $destination/

# remove_to_backup(path + 'train/cats/', path + 'train_bkup/')
# remove_to_backup(path + 'train/dogs/', path + 'train_bkup/')

#restore_backup(path + 'train_bkup', path + 'train')

# nimages = cats_copied + dogs_copied
# def move_to_test(nimages):
#     os.chdir("/home/asaeed9/work/data/2cat/train")
#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(nimages): os.rename(shuf[i], '../sample/test/' + shuf[i])
#     os.chdir("../sample/test/")
#     move("cat*.jpg")
#     %mv ../sample/test/cat*.jpg ../sample/cats/
#     %mv ../sample/test/dog*.jpg ../sample/dogs/    




#move_from_test(retrain_set, path, train_size, validation_size)


In [ ]:
model = None
model = get_test_model()
loss_score = []
accuracy_score = []
training_size = [100]

os.chdir(results_path)
files = filter(os.path.isfile, os.listdir(results_path))
files.sort(key=lambda x: os.path.getmtime(x))

for f in files:
    
    print("\nFile being processed: {}".format(f))
    
    train_size = int(f[f.find('_')+1 : f.find('.')])
    epoch = int(f[f.find('e')+1 : ])

    model.load_weights(results_path+'/ft_' + str(train_size) + '.e' + str(epoch))
    
    gen_test = image.ImageDataGenerator()
    test_batches = gen_test.flow_from_directory(path+'test', class_mode=None, target_size=(256,256), shuffle=False, batch_size=1)
    test_data = np.concatenate([test_batches.next() for i in range(test_batches.nb_sample)])
    test_labels = onehot(test_batches.classes)
    score = model.evaluate(test_data, test_labels)
    
    loss_score.append(score[0])
    accuracy_score.append(score[1])
    
    print("\nLoss:{}, Accuracy:{}".format(score[0], score[1]))


In [ ]:
model = None
model = get_test_model()
loss_score = []
accuracy_score = []
training_size = [100]

os.chdir(results_path)
files = filter(os.path.isfile, os.listdir(results_path))
files.sort(key=lambda x: os.path.getmtime(x))

for f in files:
    
    print("\nFile being processed: {}".format(f))
    
    train_size = int(f[f.find('_')+1 : f.find('.')])
    epoch = int(f[f.find('e')+1 : ])

    model.load_weights(results_path+'/ft_' + str(train_size) + '.e' + str(epoch))
    
    gen_test = image.ImageDataGenerator()
    test_batches = gen_test.flow_from_directory(path+'test', class_mode=None, target_size=(256,256), shuffle=False, batch_size=1)
    test_data = np.concatenate([test_batches.next() for i in range(test_batches.nb_sample)])
    test_labels = onehot(test_batches.classes)
    score = model.evaluate(test_data, test_labels)
    
    probs = model.predict(test_data)
    
    loss_score.append(score[0])
    accuracy_score.append(score[1])
    
    print("\nLoss:{}, Accuracy:{}".format(score[0], score[1]))
    print("\nProbabilities:{}".format(probs))


In [ ]:
test_path

In [ ]:
from PIL import Image
Image.open(test_path +  'cats/cat.6013.jpg')


In [ ]:
#probs[:10]
retrain_idx = np.where(np.logical_and(probs >=0.4, probs<=0.6))[0]
retrain_set = [test_batches.filenames[i] for i in retrain_idx]



retrain_set[:10]

In [ ]:
def get_train_model(tr_batches, epoch):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3, 256,256)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
#             Convolution2D(64,3,3, activation='relu'),
#             BatchNormalization(axis=1),
            #MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Flatten(),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(2, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    #model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=2)
    #model.optimizer.lr = 0.001
    #model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=epoch - 2)
    return model 

In [ ]:
#model = get_train_model(new_data, nepoch)
#model.load_weights(results_path + "/ft_100.e50")
#model.fit_generator(new_data, len(new_data), nb_epoch = 2)